# Topic Modelling Fundamentals by Example

---
---

In [ ]:
# Create a corpora dictionary of tokens using Gensim (corpora.Dictionary)

In [ ]:
# Convert to a bag-of-words corpus using Gensim (corpora.Dictionary.doc2bow)

## Bag of words

A bag-of-words (BoW) corpus is a _vocabulary_ of the known words in the corpus together with some _measure_ of how often they occur. The measurement may be:
* binary (presence or absence)
* count (how many times the word occurs)
* frequency (count divided by the total number of words).  

This combination of vocabulary and measurement is called a **document vector**.

#### Example

Here is a simplified example to demonstrate the principles of creating a vector from a document.

Document (20 words):

>'No room to poise the lance or bend the bow;
> But hand to hand, and man to man, they grow:'
 
 (from _The Iliad of Homer_, translated by Alexander Pope (1899)) 
  
Vocabulary of unique words (15 words):

* no
* room
* to
* poise
* the
* lance
* or
* bend
* bow
* but
* hand
* and
* man
* they
* grow

Count measurements (how many times each word appears in the document):

* no = 1
* room = 1
* to = 3
* poise = 1
* the = 2
* lance = 1
* or = 1
* bend = 1
* bow = 1
* but = 1
* hand = 2
* and = 1
* man = 2
* they = 1
* grow = 1

If we treat this vocabulary as a list with a fixed order, we can just extract the counts into a list. This is the document vector.

`[1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1]`

In order to compare other documents with this one for similarity, we could generate a document vector with the same vocabulary list for each document, or expand the vocabulary list to cover all the words in all the documents we are interested in.

#### The 'bag' in bag of wordsprocessing

In this most basic BoW model all order and location of the words is discarded. For example, it does not matter if the words 'red' and 'nose' are adjacent ('red nose'), or at the beginning or end of a sentence; BoW just treats the words individually. It is like a 'bag' of Scrabble™ tiles, where each tile is a word, all rattling around together in no particular order.

It is possible to create a BoW corpus that uses two or more adjacent words, and potentially . For example, if you measure all pairs of words in our example document (above) you might end up with a vocabulary that looks like this:

* no room
* room to
* to poise
* poise the
* the lance
* lance or
* or bend
* bend the
* the bow
* bow but
* but hand
* hand to
* to hand
* hand and
* and man
* man to
* to man
* man they
* they grow

#### n-grams

Two adjacent words together like this is known as an **bigram**. The case before where we took just one word is called a **unigram**. Three words is a **trigram** and so on. These are all special cases of **n-gram**, where _n_ is some number of words.

#### Vocabulary choice

As you may have suspected by now, the size and nature of the vocabulary you choose is vitally important. A large vocabulary will take more computational power and memory to analyse. A vocabulary with many rare words (so the count for these words is 0) creates what is called a _sparse_ vector, which has less useful information in it. Likewise, very common but largely meaningless words are often wasteful to include, for example, we would probably want to exclude a list of **stopwords**.

#### Term Frequency–Inverse Document Frequency (TF-IDF)
If you measure word frequency, highly frequent words come to dominate your results and yet they may not be as meaningful or interesting as rarer words. For example, if you are looking at articles about the history of the Moon landings, even if you have removed all the stopwords, you may well find that the words 'lunar', 'moon', 'landing', 'orbit', and 'earth' predominate. Subtle differences in topic between documents may be lost.

One way to deal with this is to use a _weighting factor_ called **TF-IDF**. A value is calculated for each word that reflects:
* Term frequency (TF) - the number of times the word appears in the document
* Document frequency (DF) - the number of documents in the corpus that contain the word

For example, if a very uncommon word is present in two documents, this word is weighted more highly than a word that is present in all documents in a corpus.


In [ ]:
# Save bag-of-words corpus object to disc using pickle (alternatives?)

## LDA

In [ ]:
# Use LDA model to find topics using Gensim (gensim.models.ldamodel.LdaModel)

In [ ]:
from gensim.corpora import Dictionary
dct = Dictionary(["máma mele maso".split(), "ema má máma".split()])
str(dct)
for item in dct.items():
    print(item)

In [ ]:
dct.doc2bow(["this", "is", "máma"])

In [ ]:
dct.doc2bow(["this", "is", "máma"], return_missing=True)

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary([text_data])
dictionary

In [ ]:
corpus = [dictionary.doc2bow(text) for text in [text_data]]
corpus[:100]

In [ ]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [ ]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

In [ ]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

In [ ]:
dictionary.save('dictionary.gensim')

Put this in somewhere?

### A Bit About Machine Learning

You may not have realised it when you started this notebook, but topic modelling is a Machine Learning (ML) method. ML is, of course, something of a hot topic...

Topic modelling is described as an **unsupervised** **classification** technique.

**Model**
... 

**Classification**

**Supervised and unsupervised techniques**

Since topic modelling is an _unsupervised_ technique, we need to spend some time evaluating the topic models that are produced. We will do this in our worked example, below.

---
---
## Summary

Blah

Blah: 

* sdfsdfsdf
* sdfsdfsdf

👌👌👌

The next notebook ...